In [1]:
import pandas as pd
import numpy as np
import itertools
import copy
import requests
import datetime
from datetime import date
import math
from bs4 import BeautifulSoup
from IPython.core.display import display, HTML
from espn_api.basketball import League
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
acronym_to_team_dict = {'ATL': 'Atlanta Hawks',
                        'BOS': 'Boston Celtics',
                        'BKN': 'Brooklyn Nets',
                        'CHA': 'Charlotte Hornets',
                        'CHI': 'Chicago Bulls',
                        'CLE': 'Cleveland Cavaliers',
                        'DAL': 'Dallas Mavericks',
                        'DEN': 'Denver Nuggets',
                        'DET': 'Detroit Pistons',
                        'GSW': 'Golden State Warriors',
                        'HOU': 'Houston Rockets',
                        'IND': 'Indiana Pacers',
                        'LAC': 'Los Angeles Clippers',
                        'LAL': 'Los Angeles Lakers',
                        'MEM': 'Memphis Grizzlies',
                        'MIA': 'Miami Heat',
                        'MIL': 'Milwaukee Bucks',
                        'MIN': 'Minnesota Timberwolves',
                        'NOP': 'New Orleans Pelicans',
                        'NYK': 'New York Knicks',
                        'OKC': 'Oklahoma City Thunder',
                        'ORL': 'Orlando Magic',
                        'PHL': 'Philadelphia 76ers',
                        'PHX': 'Phoenix Suns',
                        'POR': 'Portland Trail Blazers',
                        'SAC': 'Sacramento Kings',
                        'SAS': 'San Antonio Spurs',
                        'TOR': 'Toronto Raptors',
                        'UTA': 'Utah Jazz',
                        'WAS': 'Washignton Wizards'
                        }

team_to_acronym_dict = {'Atlanta Hawks': 'ATL',
                        'Boston Celtics': 'BOS',
                        'Brooklyn Nets': 'BKN',
                        'Charlotte Hornets': 'CHA',
                        'Chicago Bulls': 'CHI',
                        'Cleveland Cavaliers': 'CLE',
                        'Dallas Mavericks': 'DAL',
                        'Denver Nuggets': 'DEN',
                        'Detroit Pistons': 'DET',
                        'Golden State Warriors': 'GSW',
                        'Houston Rockets': 'HOU',
                        'Indiana Pacers': 'IND',
                        'Los Angeles Clippers': 'LAC',
                        'Los Angeles Lakers': 'LAL',
                        'Memphis Grizzlies': 'MEM',
                        'Miami Heat': 'MIA',
                        'Milwaukee Bucks': 'MIL',
                        'Minnesota Timberwolves': 'MIN',
                        'New Orleans Pelicans': 'NOP',
                        'New York Knicks': 'NYK',
                        'Oklahoma City Thunder': 'OKC',
                        'Orlando Magic': 'ORL',
                        'Philadelphia 76ers': 'PHL',
                        'Phoenix Suns': 'PHX',
                        'Portland Trail Blazers': 'POR',
                        'Sacramento Kings': 'SAC',
                        'San Antonio Spurs': 'SAS',
                        'Toronto Raptors': 'TOR',
                        'Utah Jazz': 'UTA',
                        'Washington Wizards': 'WAS'
                       }

In [34]:
def reading_data(file_name):
    """Loads spreadsheet
    Args:
        file_name: path of file
    Returns:
        dataframe of data
    """
    return(pd.read_csv(file_name))

def week_day_teams(schedule_df):   
    week_day_df = {}
    for index, row in schedule_df.iterrows():

        if 'Week ' + str(row['Week']) not in week_day_df.keys():
            week_day_df['Week ' + str(row['Week'])] = {}

        if row['Date'] not in week_day_df['Week ' + str(row['Week'])].keys():
            week_day_df['Week ' + str(row['Week'])][row['Date']] = []

        week_day_df['Week ' + str(row['Week'])][row['Date']].append(team_to_acronym_dict[row['Visitor']])
        week_day_df['Week ' + str(row['Week'])][row['Date']].append(team_to_acronym_dict[row['Home']])

    return week_day_df 

def daily_players(week_num, player_dict, week_day_df):
    week = 'Week ' + str(week_num)
    weekly_dict = week_day_df[week]
    day_dict = {}

    for date, teams in weekly_dict.items():
        for team in teams:
            if team not in player_dict.keys():
                continue

            for player in player_dict[team]:

                if player.injuryStatus == 'OUT':
                    continue
                    
                if date not in day_dict.keys():
                    day_dict[date] = []
                day_dict[date].append(player)
    return day_dict

def team_player_dict_creator(roster):
    player_team_dict = {}
    team_player_dict = {}

    for player in roster:
        player_team_dict[player] = player.proTeam

    for player, team in player_team_dict.items():
        if team not in team_player_dict.keys():
            team_player_dict[team] = [player]
        else:
            team_player_dict[team].append(player)
   
    return(team_player_dict)

def week_projected_points(week_num, week_day_df, team_player_dict):
    position_order_list = ['PG', 'SG', 'SF', 'PF', 'C', 'G', 'F', 'UT', 'BE']
    lineup_size = 10
    roster_size = 15
    max_guards_possible = 3
    max_forwards_possible = 3
    guard_positions = ['PG', 'SG']
    foward_positions = ['SF', 'PF']

    day_dict = daily_players(week_num, team_player_dict, week_day_df) 
    day_projected_points = {}

    for day, players in day_dict.items():
        #print(list(itertools.permutations(players)))
        max_players = len(players)
        current_num_players = 0
        roster_spots = {'PG': None, 'SG': None, 'SF': None, 'PF': None, 'C': None, 'G': None, 'F': None, 'UT': []}
        players_list = []
        total_projected_points = 0
        failed = 0
        combination_num = 0
        players_only_position = 0
        max_points = 0

        player_point_dict = {}
        for player in players:
            player_point_dict[player] = player.avg_points

        player_point_dict = {k: v for k, v in sorted(player_point_dict.items(), key=lambda item: item[1], reverse=True)} 
        players_ordered_list = list(player_point_dict.keys())

        position_player_dict = {'PG': [], 'SG': [], 'SF': [], 'PF': [], 'C': [], 'G': [], 'F': []}
        for position in ['PG', 'SG', 'SF', 'PF', 'C', 'G', 'F']:
            for player in players_ordered_list:
                if position in player.eligibleSlots:
                    position_player_dict[position].append(player)

        position_only_players = []  # players who are the only player at a position
        one_player_positions = []
        no_one_positon = False  # we will loop again if we have a success
        empty_positions = []  # no players at position
        while no_one_positon == False:

            no_one_positon = True
            for position, players in position_player_dict.items():
                if len(players) == 0 and position not in empty_positions and position not in one_player_positions:
                    empty_positions.append(position) 

                if len(players) == 1 and players[0] not in position_only_players:
                    one_player_positions.append(position)
                    roster_spots[position] = players[0]
                    total_projected_points+=players[0].avg_points
                    players_ordered_list.remove(players[0])
                    position_only_players.append(players[0])
                    players_only_position+=1
                    no_one_positon = False

                    for position2, players2 in position_player_dict.items():
                        if len(players2) == 0:
                            continue
                        if position_only_players[-1] in players2:
                            players2.remove(position_only_players[-1])
                            #position_player_dict[position2].remove(position_only_players[-1])

                if ((position == 'PG' and roster_spots['PG'] != None and roster_spots['SG'] == None and roster_spots['G'] == None and len(position_player_dict['G']) == max_guards_possible-1)
                    or (position == 'SG' and roster_spots['SG'] != None and roster_spots['PG'] == None and roster_spots['G'] == None and len(position_player_dict['G']) == max_guards_possible-1)):
                    if roster_spots['PG'] == None:
                        roster_spots['PG'] = position_player_dict['PG'][0]
                        roster_spots['G'] = position_player_dict['PG'][1]  
                        one_player_positions.append('PG')
                        one_player_positions.append('G')                            
                        total_projected_points+=position_player_dict['PG'][0].avg_points
                        total_projected_points+=position_player_dict['PG'][1].avg_points                      
                        players_ordered_list.remove(position_player_dict['PG'][0])
                        players_ordered_list.remove(position_player_dict['PG'][1])
                        position_only_players.append(position_player_dict['PG'][0])
                        position_only_players.append(position_player_dict['PG'][1])
                        players_only_position+=2
                        no_one_positon = False
                    if roster_spots['SG'] == None:
                        roster_spots['SG'] = position_player_dict['SG'][0]
                        roster_spots['G'] = position_player_dict['SG'][1]  
                        one_player_positions.append('SG')
                        one_player_positions.append('G')                            
                        total_projected_points+=position_player_dict['SG'][0].avg_points
                        total_projected_points+=position_player_dict['SG'][1].avg_points                      
                        players_ordered_list.remove(position_player_dict['SG'][0])
                        players_ordered_list.remove(position_player_dict['SG'][1])
                        position_only_players.append(position_player_dict['SG'][0])
                        position_only_players.append(position_player_dict['SG'][1])
                        players_only_position+=2
                        no_one_positon = False   

                    for position2, players2 in position_player_dict.items():
                        if len(players2) == 0:
                            continue
                        if position_only_players[-1] in players2:
                            players2.remove(position_only_players[-1]) 
                            #position_player_dict[position2].remove(position_only_players[-1])

                        if position_only_players[-2] in players2:
                            players2.remove(position_only_players[-2])
                            #position_player_dict[position2].remove(position_only_players[-2])

                if ((position == 'SF' and roster_spots['SF'] != None and roster_spots['PF'] == None and roster_spots['F'] == None and len(position_player_dict['F']) == max_guards_possible-1)
                    or (position == 'PF' and roster_spots['PF'] != None and roster_spots['SF'] == None and roster_spots['F'] == None and len(position_player_dict['F']) == max_guards_possible-1)):
                    if roster_spots['SF'] == None:
                        roster_spots['SF'] = position_player_dict['SF'][0]
                        roster_spots['F'] = position_player_dict['SF'][1]  
                        one_player_positions.append('SF')
                        one_player_positions.append('F')                            
                        total_projected_points+=position_player_dict['SF'][0].avg_points
                        total_projected_points+=position_player_dict['SF'][1].avg_points                      
                        players_ordered_list.remove(position_player_dict['SF'][0])
                        players_ordered_list.remove(position_player_dict['SF'][1])
                        position_only_players.append(position_player_dict['SF'][0])
                        position_only_players.append(position_player_dict['SF'][1])
                        players_only_position+=2
                        no_one_positon = False
                    if roster_spots['PF'] == None:
                        roster_spots['PF'] = position_player_dict['PF'][0]
                        roster_spots['F'] = position_player_dict['PF'][1]  
                        one_player_positions.append('PF')
                        one_player_positions.append('F')                            
                        total_projected_points+=position_player_dict['PF'][0].avg_points
                        total_projected_points+=position_player_dict['PF'][1].avg_points                      
                        players_ordered_list.remove(position_player_dict['PF'][0])
                        players_ordered_list.remove(position_player_dict['PF'][1])
                        position_only_players.append(position_player_dict['PF'][0])
                        position_only_players.append(position_player_dict['PF'][1])
                        players_only_position+=2
                        no_one_positon = False   

                    for position2, players2 in position_player_dict.items():
                        if len(players2) == 0:
                            continue
                        if position_only_players[-1] in players2:
                            players2.remove(position_only_players[-1]) 

                        if position_only_players[-2] in players2:
                            players2.remove(position_only_players[-2]) 

        num_empty_positions = len(empty_positions)   # Number of positions that will be empty
        """   if 'PG' in empty_positions and 'SG' in empty_positions:
            num_empty_positions+=1
        if 'SF' in empty_positions and 'PF' in empty_positions:
            num_empty_positions+=1
        #if roster_spots['PG'] != None and roster_spots['SG'] != None:   # G position will be empty
        #    print("aaaaaaaaa")
         #   num_empty_positions+=1
        #if roster_spots['SF'] != None and roster_spots['PF'] != None:   # F position will be empty
        #    print("bbbbbbbbbbbbbbbbbbb")
         #   num_empty_positions+=1
        """

        while (current_num_players + players_only_position + num_empty_positions) < min(max_players, lineup_size):
            players_ordered_list_min = []

            if failed > 0:  # If first attempt fails
                print("FAILED " + str(failed))
                if failed == 1:
                    possible_iterations = list(itertools.permutations(players_ordered_list))

                if max_players <= lineup_size:  # If number of players playing is less than or equal to starting spots
                    if failed >= len(possible_iterations):
                        tried_every_combination = 1
                        day_projected_points[day] = max_points
                        print("Tried every Combo")
                        print(day_projected_points)
                        break
                    players_ordered_list_min = possible_iterations[failed]
                else:  # If number of players playing is greater than starting spots
                    if combination_num == 0:  # If have not failed with all attmepts at original list
                        temp_list = players_ordered_list[:lineup_size]  # temporary list
                    if failed <= math.factorial(lineup_size):  # If not failed with all attempts at current list
                        players_ordered_list_min = possible_iterations[failed]
                    else:  # If have failed with all attempts at current list
                        combination_num += 1  # Will increase for each failure
                        temp_list = list(itertools.combinations(players_ordered_list, lineup_size))[combination_num]  # Going to next combination
                        players_ordered_list_min = temp_list
                        failed = 0  # Resetting fails

            else:
                players_ordered_list_min = players_ordered_list[:lineup_size]

            for player in players_ordered_list_min:
                eligible_slots = []

                for pos in position_order_list:
                    if pos in player.eligibleSlots:
                        eligible_slots.append(pos)

                for spot in eligible_slots:

                    if spot in roster_spots.keys() and spot != 'UT' and roster_spots[spot] == None:
                        roster_spots[spot] = player
                        players_list.append(player)
                        total_projected_points+=player.avg_points
                        current_num_players+=1
                        break
                    elif spot == 'UT' and len(roster_spots[spot]) < 3:
                        roster_spots[spot].append(player)
                        players_list.append(player)
                        total_projected_points+=player.avg_points
                        current_num_players+=1
                        break

            if ((current_num_players + players_only_position + num_empty_positions) < max_players and max_players <= lineup_size) or (max_players > lineup_size and (current_num_players + players_only_position + num_empty_positions) < lineup_size):
                current_num_players = 0
                players_list = []
                if max_points < total_projected_points:   # If this iteration performed the best, make a note
                    max_points = total_projected_points

                total_projected_points = 0
                roster_spots = {'PG': None, 'SG': None, 'SF': None, 'PF': None, 'C': None, 'G': None, 'F': None, 'UT': []}
                failed +=1
        if day not in day_projected_points.keys():
            day_projected_points[day] = total_projected_points

    return(day_projected_points, sum(day_projected_points.values()))

def daily_projected_points(week_num, day, team_player_dict):
    print(day)
    position_order_list = ['PG', 'SG', 'SF', 'PF', 'C', 'G', 'F', 'UT', 'BE']
    lineup_size = 10
    roster_size = 15
    max_guards_possible = 3
    max_forwards_possible = 3
    guard_positions = ['PG', 'SG']
    foward_positions = ['SF', 'PF']
    if day not in daily_players(week_num, team_player_dict, week_day_df).keys():
        return 0
    players = daily_players(week_num, team_player_dict, week_day_df)[day] 
    day_projected_points = {}

    max_players = len(players)
    current_num_players = 0
    roster_spots = {'PG': None, 'SG': None, 'SF': None, 'PF': None, 'C': None, 'G': None, 'F': None, 'UT': []}
    players_list = []
    total_projected_points = 0
    failed = 0
    combination_num = 0
    players_only_position = 0
    max_points = 0

    player_point_dict = {}
    for player in players:
        player_point_dict[player] = player.avg_points

    player_point_dict = {k: v for k, v in sorted(player_point_dict.items(), key=lambda item: item[1], reverse=True)} 
    players_ordered_list = list(player_point_dict.keys())
    print(player_point_dict)
    position_player_dict = {'PG': [], 'SG': [], 'SF': [], 'PF': [], 'C': [], 'G': [], 'F': []}
    for position in ['PG', 'SG', 'SF', 'PF', 'C', 'G', 'F']:
        for player in players_ordered_list:
            if position in player.eligibleSlots:
                position_player_dict[position].append(player)
    print(position_player_dict)
    position_only_players = []  # players who are the only player at a position
    one_player_positions = []
    no_one_positon = False  # we will loop again if we have a success
    empty_positions = []  # no players at position
    while no_one_positon == False:

        no_one_positon = True
        for position, players in position_player_dict.items():
            if len(players) == 0 and position not in empty_positions and position not in one_player_positions:
                empty_positions.append(position) 

            if len(players) == 1 and players[0] not in position_only_players:
                one_player_positions.append(position)
                roster_spots[position] = players[0]
                total_projected_points+=players[0].avg_points
                players_ordered_list.remove(players[0])
                position_only_players.append(players[0])
                players_only_position+=1
                no_one_positon = False

                for position2, players2 in position_player_dict.items():
                    if len(players2) == 0:
                        continue
                    if position_only_players[-1] in players2:
                        players2.remove(position_only_players[-1])
                        #position_player_dict[position2].remove(position_only_players[-1])

            if ((position == 'PG' and roster_spots['PG'] != None and roster_spots['SG'] == None and roster_spots['G'] == None and len(position_player_dict['G']) == max_guards_possible-1)
                or (position == 'SG' and roster_spots['SG'] != None and roster_spots['PG'] == None and roster_spots['G'] == None and len(position_player_dict['G']) == max_guards_possible-1)):
                if roster_spots['PG'] == None:
                    roster_spots['PG'] = position_player_dict['PG'][0]
                    roster_spots['G'] = position_player_dict['PG'][1]  
                    one_player_positions.append('PG')
                    one_player_positions.append('G')                            
                    total_projected_points+=position_player_dict['PG'][0].avg_points
                    total_projected_points+=position_player_dict['PG'][1].avg_points                      
                    players_ordered_list.remove(position_player_dict['PG'][0])
                    players_ordered_list.remove(position_player_dict['PG'][1])
                    position_only_players.append(position_player_dict['PG'][0])
                    position_only_players.append(position_player_dict['PG'][1])
                    players_only_position+=2
                    no_one_positon = False
                if roster_spots['SG'] == None:
                    roster_spots['SG'] = position_player_dict['SG'][0]
                    roster_spots['G'] = position_player_dict['SG'][1]  
                    one_player_positions.append('SG')
                    one_player_positions.append('G')                            
                    total_projected_points+=position_player_dict['SG'][0].avg_points
                    total_projected_points+=position_player_dict['SG'][1].avg_points                      
                    players_ordered_list.remove(position_player_dict['SG'][0])
                    players_ordered_list.remove(position_player_dict['SG'][1])
                    position_only_players.append(position_player_dict['SG'][0])
                    position_only_players.append(position_player_dict['SG'][1])
                    players_only_position+=2
                    no_one_positon = False   

                for position2, players2 in position_player_dict.items():
                    if len(players2) == 0:
                        continue
                    if position_only_players[-1] in players2:
                        players2.remove(position_only_players[-1]) 
                        #position_player_dict[position2].remove(position_only_players[-1])

                    if position_only_players[-2] in players2:
                        players2.remove(position_only_players[-2])
                        #position_player_dict[position2].remove(position_only_players[-2])

            if ((position == 'SF' and roster_spots['SF'] != None and roster_spots['PF'] == None and roster_spots['F'] == None and len(position_player_dict['F']) == max_guards_possible-1)
                or (position == 'PF' and roster_spots['PF'] != None and roster_spots['SF'] == None and roster_spots['F'] == None and len(position_player_dict['F']) == max_guards_possible-1)):
                if roster_spots['SF'] == None:
                    roster_spots['SF'] = position_player_dict['SF'][0]
                    roster_spots['F'] = position_player_dict['SF'][1]  
                    one_player_positions.append('SF')
                    one_player_positions.append('F')                            
                    total_projected_points+=position_player_dict['SF'][0].avg_points
                    total_projected_points+=position_player_dict['SF'][1].avg_points                      
                    players_ordered_list.remove(position_player_dict['SF'][0])
                    players_ordered_list.remove(position_player_dict['SF'][1])
                    position_only_players.append(position_player_dict['SF'][0])
                    position_only_players.append(position_player_dict['SF'][1])
                    players_only_position+=2
                    no_one_positon = False
                if roster_spots['PF'] == None:
                    roster_spots['PF'] = position_player_dict['PF'][0]
                    roster_spots['F'] = position_player_dict['PF'][1]  
                    one_player_positions.append('PF')
                    one_player_positions.append('F')                            
                    total_projected_points+=position_player_dict['PF'][0].avg_points
                    total_projected_points+=position_player_dict['PF'][1].avg_points                      
                    players_ordered_list.remove(position_player_dict['PF'][0])
                    players_ordered_list.remove(position_player_dict['PF'][1])
                    position_only_players.append(position_player_dict['PF'][0])
                    position_only_players.append(position_player_dict['PF'][1])
                    players_only_position+=2
                    no_one_positon = False   

                for position2, players2 in position_player_dict.items():
                    if len(players2) == 0:
                        continue
                    if position_only_players[-1] in players2:
                        players2.remove(position_only_players[-1]) 

                    if position_only_players[-2] in players2:
                        players2.remove(position_only_players[-2]) 

    num_empty_positions = len(empty_positions)   # Number of positions that will be empty
    """   if 'PG' in empty_positions and 'SG' in empty_positions:
        num_empty_positions+=1
    if 'SF' in empty_positions and 'PF' in empty_positions:
        num_empty_positions+=1
    #if roster_spots['PG'] != None and roster_spots['SG'] != None:   # G position will be empty
    #    print("aaaaaaaaa")
     #   num_empty_positions+=1
    #if roster_spots['SF'] != None and roster_spots['PF'] != None:   # F position will be empty
    #    print("bbbbbbbbbbbbbbbbbbb")
     #   num_empty_positions+=1
    """
    print("current_num " + str(current_num_players))
    print("players_only_position " + str(players_only_position))
    print("num_empty_positions " + str(num_empty_positions))
    print("max player " + str(max_players))
    while (current_num_players + players_only_position + num_empty_positions) < min(max_players, lineup_size):
        players_ordered_list_min = []

        if failed > 0:  # If first attempt fails
            print("FAILED " + str(failed))
            if failed == 1:
                possible_iterations = list(itertools.permutations(players_ordered_list))

            if max_players <= lineup_size:  # If number of players playing is less than or equal to starting spots
                if failed >= len(possible_iterations):
                    tried_every_combination = 1
                    day_projected_points[day] = max_points
                    print("Tried every Combo")
                    print(day_projected_points)
                    break
                players_ordered_list_min = possible_iterations[failed]
            else:  # If number of players playing is greater than starting spots
                if combination_num == 0:  # If have not failed with all attmepts at original list
                    temp_list = players_ordered_list[:lineup_size]  # temporary list
                if failed <= math.factorial(lineup_size):  # If not failed with all attempts at current list
                    players_ordered_list_min = possible_iterations[failed]
                else:  # If have failed with all attempts at current list
                    combination_num += 1  # Will increase for each failure
                    temp_list = list(itertools.combinations(players_ordered_list, lineup_size))[combination_num]  # Going to next combination
                    players_ordered_list_min = temp_list
                    failed = 0  # Resetting fails

        else:
            players_ordered_list_min = players_ordered_list[:lineup_size]

        for player in players_ordered_list_min:
            eligible_slots = []

            for pos in position_order_list:
                if pos in player.eligibleSlots:
                    eligible_slots.append(pos)

            for spot in eligible_slots:

                if spot in roster_spots.keys() and spot != 'UT' and roster_spots[spot] == None:
                    roster_spots[spot] = player
                    players_list.append(player)
                    total_projected_points+=player.avg_points
                    current_num_players+=1
                    break
                elif spot == 'UT' and len(roster_spots[spot]) < 3:
                    roster_spots[spot].append(player)
                    players_list.append(player)
                    total_projected_points+=player.avg_points
                    current_num_players+=1
                    break
        print("roster_spots")
        print(roster_spots)
        if ((current_num_players + players_only_position + num_empty_positions) < max_players and max_players <= lineup_size) or (max_players > lineup_size and (current_num_players + players_only_position + num_empty_positions) < lineup_size):
            current_num_players = 0
            players_list = []
            if max_points < total_projected_points:   # If this iteration performed the best, make a note
                max_points = total_projected_points

            total_projected_points = 0
            roster_spots = {'PG': None, 'SG': None, 'SF': None, 'PF': None, 'C': None, 'G': None, 'F': None, 'UT': []}
            failed +=1
    if day not in day_projected_points.keys():
        day_projected_points[day] = total_projected_points

    return(total_projected_points)


In [4]:
file_name = 'C:\\Users\\Jack\\Basketball Project\\NBA_Schedule.csv'
schedule_df = reading_data(file_name) # Retrieving Spreadsheet
league = League(league_id=1762898804, year=2023, espn_s2 = 'AEAkbmf6ntGyDFU%2BZ5MICm2WvwmwQ7SsLFomcaskbGz8VksoIPAE1D3MYbsTpcyiNBklJETozxCt1D0n3U7yat2RFOSQrW%2FqZdbWbsdxVWuRfOGKCfOfLrdDSYm2VsduCiWS0c5JLZ2zXzCg1UQqzReuFgIK8KLGyLj9%2FNe2IGgEgPqjwcqEp6PSBb6EssdzMJmuQMRgWtWIBE%2BRX1Ii%2FqLJTRTfLOchv6DK2%2FY%2BitmWtiY4Oa2n6gyF0Acup9V34nFNYB8IKmAZLpZqz3QFlTPw', swid='{A98C2FBC-429C-481F-8D11-8EDAE4526062}', debug=False)
free_agents = league.free_agents(size = 250)

In [ ]:
centers_team = []
num_c = 0
num_pg = 0
num_sg = 0
num_sf = 0
num_pf = 0

for player in league.free_agents(size = 250):
    if num_c > 15:
        break
    print(player.eligibleSlots)
    
        
    if player.eligibleSlots == ['C', 'PF/C', 'F/C', 'UT', 'BE', 'IR']:
        centers_team.append(player)
        num_c+=1
        
print(centers_team)

In [ ]:
centers_team[7].eligibleSlots = ['PG', 'G']
centers_team[6].eligibleSlots = ['SG', 'G']
centers_team[5].eligibleSlots = ['SF', 'F']
centers_team[4].eligibleSlots = ['PF', 'F']
centers_team[2].eligibleSlots = ['SG', 'G']


In [35]:
for week in range(1,15):
    for league_team in range(1,10):
        #week = 8
        print("week " + str(week))
        print("team " + str(league_team))
        team = league.teams[league_team]
        orig_roster = copy.copy(team.roster)
        week_day_df = week_day_teams(schedule_df)
        orig_weekly_points_dict, orig_weekly_points = week_projected_points(week, week_day_df, team_player_dict_creator(orig_roster))

        day_of_week = 0
        prior_points = 0

        player_value_dict = {}
        for day in week_day_df['Week ' + str(week)].keys(): 
            print(day)
            #if datetime.datetime.strptime(day, '%m/%d/%y') < datetime.datetime.today():
                #continue
            print("Making changes prior to " + day)
            if day_of_week > 0:
                prior_points = 0
                for prev_day in week_day_df['Week ' + str(week)].keys():
                    if prev_day >= day:
                        break
                    prior_points+=daily_projected_points(week, prev_day, team_player_dict_creator(orig_roster))

            day_of_week+=1
            print("Points prior to day " + str(prior_points))

            for player in team.roster:
                if player.avg_points > 25:
                    continue

                else:
                    if player not in player_value_dict.keys():
                        player_value_dict[player] = {}
                    for free_agent in free_agents:
                        if free_agent.injuryStatus != 'ACTIVE' or free_agent.avg_points < 18:
                            continue
                        else:
                            temp_roster = copy.copy(orig_roster)
                            temp_roster.remove(player)
                            temp_roster.append(free_agent)
                            temp_roster_dict = team_player_dict_creator(temp_roster)
                            additional_points = 0
                            for temp_day in week_day_df['Week ' + str(week)].keys():
                                if temp_day < day:
                                    continue

                                additional_points+=daily_projected_points(week, temp_day, temp_roster_dict)

                            total_weekly_additional_points =  prior_points + additional_points - orig_weekly_points   
                            if total_weekly_additional_points > 10:
                                if free_agent not in player_value_dict[player].keys():
                                    player_value_dict[player][free_agent] = [day, total_weekly_additional_points]
                                elif total_weekly_additional_points > player_value_dict[player][free_agent][1]:
                                    player_value_dict[player][free_agent] = [day, total_weekly_additional_points]

                                player_value_dict[player] = {k: v for k, v in sorted(player_value_dict[player].items(), key=lambda item: item[1][1], reverse = True)} 
        player_value_dict

week 1
team 1
10/18/22
Making changes prior to 10/18/22
Points prior to day 0
10/18/22
{Player(Stephen Curry): 52.61}
{'PG': [Player(Stephen Curry)], 'SG': [], 'SF': [], 'PF': [], 'C': [], 'G': [Player(Stephen Curry)], 'F': []}
current_num 0
players_only_position 1
num_empty_positions 6
max player 1
10/19/22
{Player(Domantas Sabonis): 41.64, Player(Jalen Brunson): 37.35, Player(Jaren Jackson Jr.): 36.57, Player(Fred VanVleet): 35.36, Player(Kelly Oubre Jr.): 32.85, Player(Kyle Lowry): 32.26, Player(Franz Wagner): 30.71, Player(Bruce Brown): 26.23, Player(Kevin Love): 24.87, Player(Trey Murphy III): 24.6, Player(Keegan Murray): 19.07, Player(Thaddeus Young): 15.36}
{'PG': [Player(Jalen Brunson), Player(Fred VanVleet), Player(Kyle Lowry)], 'SG': [Player(Jalen Brunson), Player(Fred VanVleet), Player(Kelly Oubre Jr.), Player(Franz Wagner), Player(Bruce Brown), Player(Trey Murphy III)], 'SF': [Player(Kelly Oubre Jr.), Player(Franz Wagner), Player(Trey Murphy III)], 'PF': [Player(Domantas Sa

{'PG': Player(Stephen Curry), 'SG': Player(Jalen Brunson), 'SF': Player(Kelly Oubre Jr.), 'PF': Player(Franz Wagner), 'C': Player(Jaren Jackson Jr.), 'G': Player(Fred VanVleet), 'F': Player(Trey Murphy III), 'UT': [Player(Kyle Lowry), Player(Bruce Brown), Player(Thaddeus Young)]}
10/22/22
{Player(Domantas Sabonis): 41.64, Player(Jaren Jackson Jr.): 36.57, Player(Fred VanVleet): 35.36, Player(Kyle Lowry): 32.26, Player(Brook Lopez): 31.68, Player(Franz Wagner): 30.71, Player(Bruce Brown): 26.23, Player(John Wall): 25.01, Player(Harrison Barnes): 24.51, Player(Thaddeus Young): 15.36}
{'PG': [Player(Fred VanVleet), Player(Kyle Lowry), Player(John Wall)], 'SG': [Player(Fred VanVleet), Player(Franz Wagner), Player(Bruce Brown)], 'SF': [Player(Franz Wagner), Player(Harrison Barnes)], 'PF': [Player(Domantas Sabonis), Player(Jaren Jackson Jr.), Player(Franz Wagner), Player(Harrison Barnes), Player(Thaddeus Young)], 'C': [Player(Domantas Sabonis), Player(Jaren Jackson Jr.), Player(Brook Lopez)]

num_empty_positions 6
max player 1
10/19/22
{Player(Domantas Sabonis): 41.64, Player(Jalen Brunson): 37.35, Player(Jaren Jackson Jr.): 36.57, Player(Fred VanVleet): 35.36, Player(Kelly Oubre Jr.): 32.85, Player(Kyle Lowry): 32.26, Player(Franz Wagner): 30.71, Player(Bruce Brown): 26.23, Player(Trey Murphy III): 24.6, Player(Nick Richards): 20.99, Player(Thaddeus Young): 15.36}
{'PG': [Player(Jalen Brunson), Player(Fred VanVleet), Player(Kyle Lowry)], 'SG': [Player(Jalen Brunson), Player(Fred VanVleet), Player(Kelly Oubre Jr.), Player(Franz Wagner), Player(Bruce Brown), Player(Trey Murphy III)], 'SF': [Player(Kelly Oubre Jr.), Player(Franz Wagner), Player(Trey Murphy III)], 'PF': [Player(Domantas Sabonis), Player(Jaren Jackson Jr.), Player(Kelly Oubre Jr.), Player(Franz Wagner), Player(Thaddeus Young)], 'C': [Player(Domantas Sabonis), Player(Jaren Jackson Jr.), Player(Nick Richards)], 'G': [Player(Jalen Brunson), Player(Fred VanVleet), Player(Kelly Oubre Jr.), Player(Kyle Lowry), Player

num_empty_positions 0
max player 11
roster_spots
{'PG': Player(Fred VanVleet), 'SG': Player(Franz Wagner), 'SF': Player(Tari Eason), 'PF': Player(Domantas Sabonis), 'C': Player(Jaren Jackson Jr.), 'G': Player(Kyle Lowry), 'F': Player(Kevin Love), 'UT': [Player(Brook Lopez), Player(Bruce Brown), Player(John Wall)]}
10/23/22
{Player(Stephen Curry): 52.61, Player(Domantas Sabonis): 41.64, Player(Kelly Oubre Jr.): 32.85, Player(John Wall): 25.01, Player(Kevin Love): 24.87, Player(Trey Murphy III): 24.6}
{'PG': [Player(Stephen Curry), Player(John Wall)], 'SG': [Player(Kelly Oubre Jr.), Player(Trey Murphy III)], 'SF': [Player(Kelly Oubre Jr.), Player(Trey Murphy III)], 'PF': [Player(Domantas Sabonis), Player(Kelly Oubre Jr.), Player(Kevin Love)], 'C': [Player(Domantas Sabonis)], 'G': [Player(Stephen Curry), Player(Kelly Oubre Jr.), Player(John Wall), Player(Trey Murphy III)], 'F': [Player(Domantas Sabonis), Player(Kelly Oubre Jr.), Player(Kevin Love), Player(Trey Murphy III)]}
current_num 0


{Player(Domantas Sabonis): 41.64, Player(Jalen Brunson): 37.35, Player(Jaren Jackson Jr.): 36.57, Player(Fred VanVleet): 35.36, Player(Kelly Oubre Jr.): 32.85, Player(Kyle Lowry): 32.26, Player(Franz Wagner): 30.71, Player(Bruce Brown): 26.23, Player(Kevin Love): 24.87, Player(Ayo Dosunmu): 20.48, Player(Thaddeus Young): 15.36}
{'PG': [Player(Jalen Brunson), Player(Fred VanVleet), Player(Kyle Lowry), Player(Ayo Dosunmu)], 'SG': [Player(Jalen Brunson), Player(Fred VanVleet), Player(Kelly Oubre Jr.), Player(Franz Wagner), Player(Bruce Brown), Player(Ayo Dosunmu)], 'SF': [Player(Kelly Oubre Jr.), Player(Franz Wagner)], 'PF': [Player(Domantas Sabonis), Player(Jaren Jackson Jr.), Player(Kelly Oubre Jr.), Player(Franz Wagner), Player(Kevin Love), Player(Thaddeus Young)], 'C': [Player(Domantas Sabonis), Player(Jaren Jackson Jr.)], 'G': [Player(Jalen Brunson), Player(Fred VanVleet), Player(Kelly Oubre Jr.), Player(Kyle Lowry), Player(Franz Wagner), Player(Bruce Brown), Player(Ayo Dosunmu)], 'F

...

num_empty_positions 0
max player 7
roster_spots
{'PG': Player(Stephen Curry), 'SG': Player(Kelly Oubre Jr.), 'SF': Player(Trey Murphy III), 'PF': Player(Kevin Love), 'C': Player(Domantas Sabonis), 'G': Player(John Wall), 'F': Player(Harrison Barnes), 'UT': []}
10/23/22
{Player(Stephen Curry): 52.61, Player(Domantas Sabonis): 41.64, Player(Kelly Oubre Jr.): 32.85, Player(John Wall): 25.01, Player(Kevin Love): 24.87, Player(Trey Murphy III): 24.6}
{'PG': [Player(Stephen Curry), Player(John Wall)], 'SG': [Player(Kelly Oubre Jr.), Player(Trey Murphy III)], 'SF': [Player(Kelly Oubre Jr.), Player(Trey Murphy III)], 'PF': [Player(Domantas Sabonis), Player(Kelly Oubre Jr.), Player(Kevin Love)], 'C': [Player(Domantas Sabonis)], 'G': [Player(Stephen Curry), Player(Kelly Oubre Jr.), Player(John Wall), Player(Trey Murphy III)], 'F': [Player(Domantas Sabonis), Player(Kelly Oubre Jr.), Player(Kevin Love), Player(Trey Murphy III)]}
current_num 0
players_only_position 1
num_empty_positions 0
max playe

{Player(Stephen Curry): 52.61, Player(Domantas Sabonis): 41.64, Player(Kelly Oubre Jr.): 32.85, Player(John Wall): 25.01, Player(Kevin Love): 24.87, Player(Trey Murphy III): 24.6, Player(Nick Richards): 20.99}
{'PG': [Player(Stephen Curry), Player(John Wall)], 'SG': [Player(Kelly Oubre Jr.), Player(Trey Murphy III)], 'SF': [Player(Kelly Oubre Jr.), Player(Trey Murphy III)], 'PF': [Player(Domantas Sabonis), Player(Kelly Oubre Jr.), Player(Kevin Love)], 'C': [Player(Domantas Sabonis), Player(Nick Richards)], 'G': [Player(Stephen Curry), Player(Kelly Oubre Jr.), Player(John Wall), Player(Trey Murphy III)], 'F': [Player(Domantas Sabonis), Player(Kelly Oubre Jr.), Player(Kevin Love), Player(Trey Murphy III)]}
current_num 0
players_only_position 0
num_empty_positions 0
max player 7
roster_spots
{'PG': Player(Stephen Curry), 'SG': Player(Kelly Oubre Jr.), 'SF': Player(Trey Murphy III), 'PF': Player(Domantas Sabonis), 'C': Player(Nick Richards), 'G': Player(John Wall), 'F': Player(Kevin Love),

10/18/22
Making changes prior to 10/18/22
Points prior to day 0
10/18/22
{Player(Marcus Smart): 29.13}
{'PG': [Player(Marcus Smart)], 'SG': [Player(Marcus Smart)], 'SF': [], 'PF': [], 'C': [], 'G': [Player(Marcus Smart)], 'F': []}
current_num 0
players_only_position 1
num_empty_positions 6
max player 1
10/19/22
{Player(Ja Morant): 47.76, Player(Bam Adebayo): 39.19, Player(Anthony Edwards): 37.9, Player(Anfernee Simons): 36.94, Player(Myles Turner): 35.69, Player(Buddy Hield): 28.53, Player(Killian Hayes): 20.03, Player(Tyus Jones): 20.01, Player(Onyeka Okongwu): 19.12, Player(Keegan Murray): 19.07}
{'PG': [Player(Ja Morant), Player(Anfernee Simons), Player(Killian Hayes), Player(Tyus Jones)], 'SG': [Player(Anthony Edwards), Player(Anfernee Simons), Player(Buddy Hield)], 'SF': [Player(Anthony Edwards), Player(Buddy Hield)], 'PF': [Player(Bam Adebayo), Player(Myles Turner), Player(Keegan Murray)], 'C': [Player(Bam Adebayo), Player(Myles Turner), Player(Onyeka Okongwu)], 'G': [Player(Ja M

10/22/22
{Player(Ja Morant): 47.76, Player(Bam Adebayo): 39.19, Player(Myles Turner): 35.69, Player(Marcus Smart): 29.13, Player(Buddy Hield): 28.53, Player(Jevon Carter): 20.75, Player(Killian Hayes): 20.03, Player(Tyus Jones): 20.01, Player(Keegan Murray): 19.07}
{'PG': [Player(Ja Morant), Player(Marcus Smart), Player(Jevon Carter), Player(Killian Hayes), Player(Tyus Jones)], 'SG': [Player(Marcus Smart), Player(Buddy Hield)], 'SF': [Player(Buddy Hield)], 'PF': [Player(Bam Adebayo), Player(Myles Turner), Player(Keegan Murray)], 'C': [Player(Bam Adebayo), Player(Myles Turner)], 'G': [Player(Ja Morant), Player(Marcus Smart), Player(Buddy Hield), Player(Jevon Carter), Player(Killian Hayes), Player(Tyus Jones)], 'F': [Player(Bam Adebayo), Player(Myles Turner), Player(Buddy Hield), Player(Keegan Murray)]}
current_num 0
players_only_position 2
num_empty_positions 0
max player 9
roster_spots
{'PG': Player(Ja Morant), 'SG': Player(Marcus Smart), 'SF': Player(Buddy Hield), 'PF': Player(Bam Ade

KeyboardInterrupt: 

In [ ]:
tyus     pg     pg
ja       pg     ut
hayes    pg     ut
simons   pg/sg  sg
hield    sg/sf  sf
edwards  sg/sf  g
barnes   sf/pf  ut
turner   pf/c   pf
adebayo  pf/c   f
okungwu  c      c


In [27]:
for league_team in range(0,11):
    print(league.teams[league_team])

Team(Curry got Carried)
Team(Team Tsang)
Team(Put That  Sh*t On)
Team(Ethan's Hospital)
Team(Curry's  Achille's Next)
Team(Jack's Slam Dunk Team)
Team(So Much Pain)
Team(Coronavirus Chris)
Team(Team Haghiri)
Team(JP BAG SZN)


IndexError: list index out of range

In [ ]:
position_order_list = ['PG', 'SG', 'SF', 'PF', 'C', 'G', 'F', 'UT', 'BE']
    lineup_size = 10
    roster_size = 15
    max_guards_possible = 3
    max_forwards_possible = 3
    guard_positions = ['PG', 'SG']
    foward_positions = ['SF', 'PF']
    if day not in daily_players(week_num, team_player_dict, week_day_df).keys():
        return 0
    players = daily_players(week_num, team_player_dict, week_day_df)[day] 
    day_projected_points = {}


    #print(list(itertools.permutations(players)))
    max_players = len(players)
    current_num_players = 0
    roster_spots = {'PG': None, 'SG': None, 'SF': None, 'PF': None, 'C': None, 'G': None, 'F': None, 'UT': []}
    players_list = []
    total_projected_points = 0
    failed = 0
    combination_num = 0
    players_only_position = 0
    max_points = 0

    player_point_dict = {}
    for player in players:
        player_point_dict[player] = player.avg_points

    player_point_dict = {k: v for k, v in sorted(player_point_dict.items(), key=lambda item: item[1], reverse=True)} 
    players_ordered_list = list(player_point_dict.keys())

    position_player_dict = {'PG': [], 'SG': [], 'SF': [], 'PF': [], 'C': [], 'G': [], 'F': []}
    for position in ['PG', 'SG', 'SF', 'PF', 'C', 'G', 'F']:
        for player in players_ordered_list:
            if position in player.eligibleSlots:
                position_player_dict[position].append(player)

    position_only_players = []  # players who are the only player at a position
    one_player_positions = []
    no_one_positon = False  # we will loop again if we have a success
    empty_positions = []  # no players at position
    while no_one_positon == False:

        no_one_positon = True
        for position, players in position_player_dict.items():
            if len(players) == 0 and position not in empty_positions and position not in one_player_positions:
                empty_positions.append(position) 

            if len(players) == 1 and players[0] not in position_only_players:
                one_player_positions.append(position)
                roster_spots[position] = players[0]
                total_projected_points+=players[0].avg_points
                players_ordered_list.remove(players[0])
                position_only_players.append(players[0])
                players_only_position+=1
                no_one_positon = False

                for position2, players2 in position_player_dict.items():
                    if len(players2) == 0:
                        continue
                    if position_only_players[-1] in players2:
                        players2.remove(position_only_players[-1])
                        #position_player_dict[position2].remove(position_only_players[-1])

            if ((position == 'PG' and roster_spots['PG'] != None and roster_spots['SG'] == None and roster_spots['G'] == None and len(position_player_dict['G']) == max_guards_possible-1)
                or (position == 'SG' and roster_spots['SG'] != None and roster_spots['PG'] == None and roster_spots['G'] == None and len(position_player_dict['G']) == max_guards_possible-1)):
                if roster_spots['PG'] == None:
                    roster_spots['PG'] = position_player_dict['PG'][0]
                    roster_spots['G'] = position_player_dict['PG'][1]  
                    one_player_positions.append('PG')
                    one_player_positions.append('G')                            
                    total_projected_points+=position_player_dict['PG'][0].avg_points
                    total_projected_points+=position_player_dict['PG'][1].avg_points                      
                    players_ordered_list.remove(position_player_dict['PG'][0])
                    players_ordered_list.remove(position_player_dict['PG'][1])
                    position_only_players.append(position_player_dict['PG'][0])
                    position_only_players.append(position_player_dict['PG'][1])
                    players_only_position+=2
                    no_one_positon = False
                if roster_spots['SG'] == None:
                    roster_spots['SG'] = position_player_dict['SG'][0]
                    roster_spots['G'] = position_player_dict['SG'][1]  
                    one_player_positions.append('SG')
                    one_player_positions.append('G')                            
                    total_projected_points+=position_player_dict['SG'][0].avg_points
                    total_projected_points+=position_player_dict['SG'][1].avg_points                      
                    players_ordered_list.remove(position_player_dict['SG'][0])
                    players_ordered_list.remove(position_player_dict['SG'][1])
                    position_only_players.append(position_player_dict['SG'][0])
                    position_only_players.append(position_player_dict['SG'][1])
                    players_only_position+=2
                    no_one_positon = False   

                for position2, players2 in position_player_dict.items():
                    if len(players2) == 0:
                        continue
                    if position_only_players[-1] in players2:
                        players2.remove(position_only_players[-1]) 
                        #position_player_dict[position2].remove(position_only_players[-1])

                    if position_only_players[-2] in players2:
                        players2.remove(position_only_players[-2])
                        #position_player_dict[position2].remove(position_only_players[-2])

            if ((position == 'SF' and roster_spots['SF'] != None and roster_spots['PF'] == None and roster_spots['F'] == None and len(position_player_dict['F']) == max_guards_possible-1)
                or (position == 'PF' and roster_spots['PF'] != None and roster_spots['SF'] == None and roster_spots['F'] == None and len(position_player_dict['F']) == max_guards_possible-1)):
                if roster_spots['SF'] == None:
                    roster_spots['SF'] = position_player_dict['SF'][0]
                    roster_spots['F'] = position_player_dict['SF'][1]  
                    one_player_positions.append('SF')
                    one_player_positions.append('F')                            
                    total_projected_points+=position_player_dict['SF'][0].avg_points
                    total_projected_points+=position_player_dict['SF'][1].avg_points                      
                    players_ordered_list.remove(position_player_dict['SF'][0])
                    players_ordered_list.remove(position_player_dict['SF'][1])
                    position_only_players.append(position_player_dict['SF'][0])
                    position_only_players.append(position_player_dict['SF'][1])
                    players_only_position+=2
                    no_one_positon = False
                if roster_spots['PF'] == None:
                    roster_spots['PF'] = position_player_dict['PF'][0]
                    roster_spots['F'] = position_player_dict['PF'][1]  
                    one_player_positions.append('PF')
                    one_player_positions.append('F')                            
                    total_projected_points+=position_player_dict['PF'][0].avg_points
                    total_projected_points+=position_player_dict['PF'][1].avg_points                      
                    players_ordered_list.remove(position_player_dict['PF'][0])
                    players_ordered_list.remove(position_player_dict['PF'][1])
                    position_only_players.append(position_player_dict['PF'][0])
                    position_only_players.append(position_player_dict['PF'][1])
                    players_only_position+=2
                    no_one_positon = False   

                for position2, players2 in position_player_dict.items():
                    if len(players2) == 0:
                        continue
                    if position_only_players[-1] in players2:
                        players2.remove(position_only_players[-1]) 

                    if position_only_players[-2] in players2:
                        players2.remove(position_only_players[-2]) 

    num_empty_positions = len(empty_positions)   # Number of positions that will be empty
    """   if 'PG' in empty_positions and 'SG' in empty_positions:
        num_empty_positions+=1
    if 'SF' in empty_positions and 'PF' in empty_positions:
        num_empty_positions+=1
    #if roster_spots['PG'] != None and roster_spots['SG'] != None:   # G position will be empty
    #    print("aaaaaaaaa")
     #   num_empty_positions+=1
    #if roster_spots['SF'] != None and roster_spots['PF'] != None:   # F position will be empty
    #    print("bbbbbbbbbbbbbbbbbbb")
     #   num_empty_positions+=1
    """

    while (current_num_players + players_only_position + num_empty_positions) < min(max_players, lineup_size):
        players_ordered_list_min = []

        if failed > 0:  # If first attempt fails
            print("FAILED " + str(failed))
            if failed == 1:
                possible_iterations = list(itertools.permutations(players_ordered_list))

            if max_players <= lineup_size:  # If number of players playing is less than or equal to starting spots
                if failed >= len(possible_iterations):
                    tried_every_combination = 1
                    day_projected_points[day] = max_points
                    print("Tried every Combo")
                    print(day_projected_points)
                    break
                players_ordered_list_min = possible_iterations[failed]
            else:  # If number of players playing is greater than starting spots
                if combination_num == 0:  # If have not failed with all attmepts at original list
                    temp_list = players_ordered_list[:lineup_size]  # temporary list
                if failed <= math.factorial(lineup_size):  # If not failed with all attempts at current list
                    players_ordered_list_min = possible_iterations[failed]
                else:  # If have failed with all attempts at current list
                    combination_num += 1  # Will increase for each failure
                    temp_list = list(itertools.combinations(players_ordered_list, lineup_size))[combination_num]  # Going to next combination
                    players_ordered_list_min = temp_list
                    failed = 0  # Resetting fails

        else:
            players_ordered_list_min = players_ordered_list[:lineup_size]

        for player in players_ordered_list_min:
            eligible_slots = []

            for pos in position_order_list:
                if pos in player.eligibleSlots:
                    eligible_slots.append(pos)

            for spot in eligible_slots:

                if spot in roster_spots.keys() and spot != 'UT' and roster_spots[spot] == None:
                    roster_spots[spot] = player
                    players_list.append(player)
                    total_projected_points+=player.avg_points
                    current_num_players+=1
                    break
                elif spot == 'UT' and len(roster_spots[spot]) < 3:
                    roster_spots[spot].append(player)
                    players_list.append(player)
                    total_projected_points+=player.avg_points
                    current_num_players+=1
                    break

        if ((current_num_players + players_only_position + num_empty_positions) < max_players and max_players <= lineup_size) or (max_players > lineup_size and (current_num_players + players_only_position + num_empty_positions) < lineup_size):
            current_num_players = 0
            players_list = []
            if max_points < total_projected_points:   # If this iteration performed the best, make a note
                max_points = total_projected_points

            total_projected_points = 0
            roster_spots = {'PG': None, 'SG': None, 'SF': None, 'PF': None, 'C': None, 'G': None, 'F': None, 'UT': []}
            failed +=1
    if day not in day_projected_points.keys():
        day_projected_points[day] = total_projected_points

    return(total_projected_points)

In [13]:

for el in list(itertools.permutations([1,2,3,4,5,6,7,8,9,10])):
    xx = 1